# Workflows in Agent Framework

**AI Agents** are dynamic - the LLM decides what steps to take at runtime based on context and available tools. Use when you need flexibility, reasoning, and open-ended problem solving.

**Workflows** are predefined sequences with explicit execution paths. The structure is fixed at design time, but can include conditional branching and parallel execution. Use when you need predictability, auditability, or coordination across multiple agents and external systems.

Workflows can contain agents as components - giving you LLM intelligence within a controlled structure.

### Core Concepts
1. **Executors**: Processing units that receive messages, perform tasks, and produce outputs. Can be agents or custom logic.
2. **Edges**: Connections between executors that control message flow. Support conditional routing.
3. **Workflows**: Directed graphs of executors and edges with a defined start point.

### Setup

First, let's validate our environment to ensure all required variables are configured.

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

from workshop_utils import validate_env

validate_env()

### Basic Executor Structure
Executors are the building blocks that process messages. They inherit from `Executor` and use the `@handler` decorator to define message handlers.

**Handler signature**: `(self, input: T_In, ctx: WorkflowContext[T_Out]) -> None`
- First param: the typed input message
- Second param: context for sending outputs downstream

#### Pattern 1: Class-based Executor

In [ ]:
from agent_framework import Executor, WorkflowContext, handler

class UpperCase(Executor):
    def __init__(self):
        super().__init__(id="upper_case")
    
    @handler
    async def handle(self, text: str, ctx: WorkflowContext[str]) -> None:
        await ctx.send_message(text.upper())

#### Pattern 2: Function-based Executor
For simple, stateless executors, use the `@executor` decorator on an async function.

In [ ]:
from agent_framework import executor, WorkflowContext

@executor(id="upper_case_fn")
async def upper_case_fn(text: str, ctx: WorkflowContext[str]) -> None:
    await ctx.send_message(text.upper())

### WorkflowContext Methods
`WorkflowContext` provides two key methods for handlers:

| Method | Purpose | Context Type |
|--------|---------|-------------|
| `send_message(value)` | Forward to downstream executors | `WorkflowContext[T_Out]` |
| `yield_output(value)` | Emit as final workflow output | `WorkflowContext[Never, T_WorkflowOut]` |

If a handler does neither, use `WorkflowContext` with no type params.

In [ ]:
from agent_framework import Executor, WorkflowContext, handler
from typing_extensions import Never

class ExampleExecutor(Executor):
    def __init__(self):
        super().__init__(id="example")
    
    @handler
    async def forward_to_next(self, msg: str, ctx: WorkflowContext[str]) -> None:
        """Send to downstream executor."""
        await ctx.send_message(msg + " processed")

    @handler
    async def emit_final_output(self, msg: str, ctx: WorkflowContext[Never, str]) -> None:
        """Yield as workflow output (terminal node)."""
        await ctx.yield_output(msg + " done")

    @handler
    async def side_effect_only(self, msg: str, ctx: WorkflowContext) -> None:
        """No output - just perform side effects."""
        print(f"Received: {msg}")

### Edge Patterns
Edges define how messages flow between executors. The framework supports:

- **Direct**: Simple one-to-one connections
- **Conditional**: Route based on message content
- **Fan-out**: One source to multiple targets
- **Fan-in**: Multiple sources to one target

Let's see direct and conditional edges in action:

In [ ]:
from agent_framework import WorkflowBuilder, WorkflowContext, executor, WorkflowOutputEvent
from typing_extensions import Never

@executor(id="upper_case")
async def upper_case(text: str, ctx: WorkflowContext[str]) -> None:
    await ctx.send_message(text.upper())

@executor(id="reverse_text")
async def reverse_text(text: str, ctx: WorkflowContext[Never, str]) -> None:
    await ctx.yield_output(f"Reversed: {text[::-1]}")

@executor(id="add_exclamation")
async def add_exclamation(text: str, ctx: WorkflowContext[Never, str]) -> None:
    await ctx.yield_output(f"Excited: {text}!!!")

# Build workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .add_edge(upper_case, reverse_text)  # Direct: always executes
    .add_edge(upper_case, add_exclamation, condition=lambda msg: len(msg) > 5)  # Conditional
    .set_start_executor(upper_case)
    .build()
)

# Test with short input (only reverse_text runs)
print("Input: 'hi'")
events = await workflow.run("hi")
print(f"Outputs: {events.get_outputs()}")

print()

# Test with long input (both reverse_text and add_exclamation run)
print("Input: 'hello world'")
events = await workflow.run("hello world")
print(f"Outputs: {events.get_outputs()}")

### Workflows
A **Workflow** connects executors and runs them. When you call `workflow.run(input)`, it sends your input to the start executor, which processes it and passes results to connected executors via edges, continuing until no more work remains.

#### Example: Simple Sequential Workflow
A two-step workflow: uppercase → reverse.

In [ ]:
from agent_framework import Executor, WorkflowBuilder, WorkflowContext, executor, handler
from typing_extensions import Never

class UpperCase(Executor):
    def __init__(self):
        super().__init__(id="upper_case")
    
    @handler
    async def handle(self, text: str, ctx: WorkflowContext[str]) -> None:
        await ctx.send_message(text.upper())

@executor(id="reverse_text")
async def reverse_text(text: str, ctx: WorkflowContext[Never, str]) -> None:
    await ctx.yield_output(text[::-1])



#### Example: Using Built-In Agent Executor

In this step, the agents we add to the workflow will be instantiated using the `AzureOpenAIChatClient` class that we already used earlier in *Section 01.1*:

In [ ]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

load_dotenv()
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

chat_client=AzureOpenAIChatClient(
        endpoint=endpoint,
        api_key=api_key,
        api_version=api_version,
        deployment_name=deployment,
    )

When you add an agent to a workflow:

- It behaves like an executor but runs with its own instructions and context.
- You can chain multiple agents together to create collaborative flows (e.g., Writer → Reviewer).
- The workflow orchestrates their interaction, passing messages and collecting outputs.

In the following example, two agents are created:
-  `writer_agent` generates and edits content and `reviewer_agent` reviews content and provides feedback
- The workflow setup uses `WorkflowBuilder` to set the start executor as the writer agent, add an edge from the writer to the reviewer and builds the workflow graph.

In [ ]:
from agent_framework import AgentRunEvent, WorkflowBuilder

async def main():
    """Build and run a simple two node agent workflow: Writer then Reviewer."""
    
    # Define agent factory functions for proper state isolation.
    # Each build() will create fresh agent instances, preventing conversation leakage.
    def create_writer_agent():
        return chat_client.as_agent(
            instructions=(
                "You are an excellent content writer. You create new content and edit contents based on the feedback."
            ),
            name="writer",
        )

    def create_reviewer_agent():
        return chat_client.as_agent(
            instructions=(
                "You are an excellent content reviewer."
                "Provide actionable feedback to the writer about the provided content."
                "Provide the feedback in the most concise manner possible."
            ),
            name="reviewer",
        )

    # Build workflow using registered agent names for state isolation
    workflow = (
        WorkflowBuilder()
        .register_agent(factory_func=create_writer_agent, name="writer")
        .register_agent(factory_func=create_reviewer_agent, name="reviewer")
        .set_start_executor("writer")
        .add_edge("writer", "reviewer")
        .build()
    )

    # Run the workflow with the user's initial message. For clarity, use run (non streaming) and print the terminal event.
    events = await workflow.run("Create a slogan for a new electric SUV that is affordable and fun to drive.")

    # Print detailed agent run events
    for event in events:
        if isinstance(event, AgentRunEvent):
            print(f"\nExecutor: {event.executor_id}")
            print(f"Output:\n{event.data}")
            print("-" * 40)

    # Print final workflow state
    print(f"Workflow State: {events.get_final_state()}")
    print("=" * 60)

await main()

### Exercise - Create a Simple Multi-Agent Workflow
In this exercise, you will design your own workflow by:

- Adding another agent to the process.
- Creating a scenario that reflects a real-world business process or planning task.
- Feel free to experiment with other concepts we have introduced so far:
    - Create an executor that performs deterministic tasks (sentiment check, calculations, pre-processing)
    - Add simple branching logic (execute two agent in parallel, for example)
    - Handle events or add simple middleware for observability

In [ ]:
# EXERCISE: Build Your Custom Workflow

# TODO: Define your scenario
# Example: "Marketing campaign planning" or "Business case validation"
scenario = ""


# TODO: Define agent factory functions for proper state isolation
# def create_agent1():
#     return chat_client.as_agent(instructions="...", name="agent1")
#
# def create_agent2():
#     return chat_client.as_agent(instructions="...", name="agent2")


# TODO (Optional): Define a custom executor
# - Add logic for validation, filtering, counting, or transformation
# - Use @handler and WorkflowContext to send messages


# TODO: Build the workflow using WorkflowBuilder with registered agents
# workflow = (
#     WorkflowBuilder()
#     .register_agent(factory_func=create_agent1, name="agent1")
#     .register_agent(factory_func=create_agent2, name="agent2")
#     .set_start_executor("agent1")
#     .add_edge("agent1", "agent2")
#     .build()
# )


# TODO: Run the workflow
# events = await workflow.run("Your task here")


# TODO: Print agent outputs and final workflow state
# for event in events:
#     if isinstance(event, AgentRunEvent):
#         print(f"Executor: {event.executor_id}")
#         print(f"Output: {event.data}")


<details>
  <summary>See the solution</summary>
  
  ```python
from agent_framework import AgentRunEvent, WorkflowBuilder

async def main():
    """Build and run a three-node agent workflow: Analyst → Finance → Approval."""

    # Define agent factory functions for proper state isolation
    def create_analyst_agent():
        return chat_client.as_agent(
            instructions=(
                "You are a business analyst. Draft a short business case based on the provided idea. "
            ),
            name="analyst",
        )

    def create_finance_agent():
        return chat_client.as_agent(
            instructions=(
                "You are a finance expert. Review the business case and highlight any financial risks or constraints."
            ),
            name="finance",
        )

    def create_approval_agent():
        return chat_client.as_agent(
            instructions=(
                "You are a senior manager. Decide whether to approve the business case based on strategic alignment. "
                "Respond with 'Approved' or 'Needs Revision' and provide reasoning."
            ),
            name="approval",
        )

    # Build workflow using registered names for state isolation
    workflow = (
        WorkflowBuilder()
        .register_agent(factory_func=create_analyst_agent, name="analyst")
        .register_agent(factory_func=create_finance_agent, name="finance")
        .register_agent(factory_func=create_approval_agent, name="approval")
        .set_start_executor("analyst")
        .add_edge("analyst", "finance")
        .add_edge("finance", "approval")
        .build()
    )


    events = await workflow.run("Develop a business case for implementing an AI-driven customer support service for handling complaints.")


    for event in events:
        if isinstance(event, AgentRunEvent):
            print(f"\nExecutor: {event.executor_id}")
            print(f"Output:\n{event.data}")
            print("-" * 40)

    print(f"Workflow State: {events.get_final_state()}")
    print("=" * 60)


await main()
```
</details>

### Example - Using Agents as Function Tools

In MAF, you can use agents as function tools by calling `.as_tool()` on an agent and providing it as a tool to another agent.
In this case, an agent's reasoning capability is wrapped so other agents can call it as part of their execution. This is useful for agent-to-agent collaboration - for example, one agent can delegate tasks to another specialised agent.

You can also customize the description, name and argument name when converting agent to a tool:
```python
refund_tool = refund_agent.as_tool(
    name="RefundAgent",
    description="Address and act on customer inquiries regarding refunds for damaged and returned items",
    arg_name="refund_inquiry",
)
```

In [ ]:
import logging
from agent_framework import function_middleware
from collections.abc import Awaitable, Callable
from agent_framework import (
    FunctionInvocationContext
)

async def function_invocation_mw(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """A filter that will be called for each function call in the response"""
    print(f"---> Agent [{context.function.name}] called with messages: {context.arguments}")

    await next(context)
    
    print(f"---> Response from agent [{context.function.name}]: {context.result}")
    
    
refund_agent = chat_client.as_agent(
    name="refund_agent",
    instructions=(
        "You specialize in addressing customer inquiries regarding refunds for broken or returned products.\n"
        "For example, if required, you can refund a customer the price of a product, given a successful return.\n "
        "For this, you'll need the order id and for sake of now, just always say that the payment has been issued. "
        ),
)
logistics_agent = chat_client.as_agent(
    name="logistics_agent",
    instructions=(
        "You specialize in handling logistics for product shipments and returns.\n"
        "You can create an order for customer (requires customer id and product id(s)).\n"
        "You can also schedule the return of a product from a customer (requires order id and pickup date). "
        ),
)

customer_agent = chat_client.as_agent(
    name="customer_agent",
    instructions=(
        "Your role is triage the customer inquiries to the appropriate agent.\n"
        "When a user returns a product, schedule the logistics and refund.\n"
        "When a user wants to place an order, use the logistics agent.\n"
        ),
    tools=[refund_agent.as_tool(), logistics_agent.as_tool()], # Add the other two agents as tools
    middleware=[function_invocation_mw],
)

thread = customer_agent.get_new_thread()

messages = [
    "hi, i want to return an order",
    "the order id is 123 and please pick it up on May 10, 2025"
]


for message in messages:
    print("*** User:", message)
    response = await customer_agent.run(message, thread=thread)
    print("*** Agent:", response.text)


### Workflows as Agents
You can also turn workflows into MAF agents and interact with the workflow as if it were an agent. This allows to integrate workflows with APIs that support the agent interface and create more powerful agents.

To create an agent out of any workflow, use the `as_agent()` method:
```python
workflow_agent = workflow.as_agent(name="Workflow Agent")
```
You can use the workflow agent just as any other MAF agent.

### Example - Wrap agents in custom executors + stream workflow events

The following example builds a two‑node workflow using custom executors that each “own” a ChatAgent. A Writer agent generates content; a Reviewer agent refines and finalizes it. 
The workflow is run with streaming, so you can observe status and data events as they happen. 

Key objectives:
- **Learn how to wrap agents as executors** - Create role‑specific agents, attach them to workflow nodes by subclassing `Executor` and using `@handler`.
- **Type‑safe data flow with `WorkflowContext`** - Model contracts between nodes to explicitly define what each executor can receive and send.
- **Event‑driven observability** - Listen to the events emitted during execution to track status changes, capture live outputs and log errors.

In [ ]:
from agent_framework import (
    ChatAgent,
    ChatMessage,
    Executor,
    ExecutorFailedEvent,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowFailedEvent,
    WorkflowOutputEvent,
    WorkflowRunState,
    WorkflowStatusEvent,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from typing_extensions import Never


class Writer(Executor):
    """Custom executor that owns a writer agent."""

    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "writer"):

        self.agent = chat_client.as_agent(
            instructions=(
                "You are an excellent content writer. You create new content and edit contents based on the feedback."
            ),
        )
        # Associate this agent with the executor node. The base Executor stores it on self.agent.
        super().__init__(id=id)

    @handler
    # Executor receives a single ChatMessage and sends downstream a list[ChatMessage] using ctx.send_message(messages).
    async def handle(self, message: ChatMessage, ctx: WorkflowContext[list[ChatMessage]]) -> None:
        """Generate content and forward the updated conversation."""
        # Start the conversation with the incoming user message.
        messages: list[ChatMessage] = [message]
        # Run the agent and extend the conversation with the agent's messages.
        response = await self.agent.run(messages)
        messages.extend(response.messages)

        await ctx.send_message(messages)


class Reviewer(Executor):
    """Custom executor that owns a review agent and completes the workflow."""

    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "reviewer"):

        self.agent = chat_client.as_agent(
            instructions=(
                "You are an excellent content reviewer. You review the content and provide feedback to the writer."
            ),
        )
        super().__init__(id=id)

    @handler
    async def handle(self, messages: list[ChatMessage], ctx: WorkflowContext[Never, str]) -> None:

        response = await self.agent.run(messages)
        await ctx.yield_output(response.text)


async def main():
    """Build the two node workflow and run it with streaming to observe events."""

    # Use factory functions for proper state isolation.
    # This ensures each build() creates fresh executor instances with their own agent state.
    def create_writer():
        return Writer(chat_client)
    
    def create_reviewer():
        return Reviewer(chat_client)

    # Build workflow using registered names
    workflow = (
        WorkflowBuilder()
        .register_executor(factory_func=create_writer, name="writer")
        .register_executor(factory_func=create_reviewer, name="reviewer")
        .set_start_executor("writer")
        .add_edge("writer", "reviewer")
        .build()
    )

    # This surfaces executor events, workflow outputs, run-state changes, and errors.
    async for event in workflow.run_stream(
        ChatMessage(role="user", text="Create a slogan for a new electric SUV that is affordable and fun to drive.")
    ):
        if isinstance(event, WorkflowStatusEvent):
            prefix = f"State ({event.origin.value}): "
            if event.state == WorkflowRunState.IN_PROGRESS:
                print(prefix + "IN_PROGRESS")
            elif event.state == WorkflowRunState.IN_PROGRESS_PENDING_REQUESTS:
                print(prefix + "IN_PROGRESS_PENDING_REQUESTS (requests in flight)")
            elif event.state == WorkflowRunState.IDLE:
                print(prefix + "IDLE (no active work)")
            elif event.state == WorkflowRunState.IDLE_WITH_PENDING_REQUESTS:
                print(prefix + "IDLE_WITH_PENDING_REQUESTS (prompt user or UI now)")
            else:
                print(prefix + str(event.state))
        elif isinstance(event, WorkflowOutputEvent):
            print(f"Workflow output ({event.origin.value}): {event.data}")
        elif isinstance(event, ExecutorFailedEvent):
            print(
                f"Executor failed ({event.origin.value}): "
                f"{event.executor_id} {event.details.error_type}: {event.details.message}"
            )
        elif isinstance(event, WorkflowFailedEvent):
            details = event.details
            print(f"Workflow failed ({event.origin.value}): {details.error_type}: {details.message}")
        else:
            # Fallback for other events that weren't explicitly handled
            print(f"{event.__class__.__name__} ({event.origin.value}): {event}")


await main()

### Exercise: Implement Your Own Workflow Scenario

Now that you’ve explored workflow basics, agents, and seen how custom executors and streaming events work, it’s time to design your own scenario. This exercise will help you apply everything you’ve learned and experiment with advanced features.

**Goal** - create a workflow that:

- Includes multiple agents with complementary roles.
- Wraps agents in custom executors for flexibility.
- Uses WorkflowContext to model type-safe data flow.
- Leverages function tools for richer capabilities.
- Implements rich logging for observability.
- Experiments with prompt variations to influence behaviour.

In [ ]:
# Your implementation here

# TODO: Define your scenario and specialized agents with complementary roles
# - Ensure instructions are detailed and clear
# agent1 = chat_client.as_agent(instructions="...", name="...")

# TODO: Wrap agents in custom executors or turn them into tools
# class MyExecutor(Executor):
#     @handler
#     async def handle(self, msg: ChatMessage, ctx: WorkflowContext[...]):
#         ...

# TODO: Configure WorkflowContext for type-safe data flow
# - Use appropriate type parameters: WorkflowContext[T_Out] or WorkflowContext[Never, T_Yield]

# TODO (Optional): Add function tools to enhance agent capabilities

# TODO: Build workflow and implement streaming with comprehensive logging
# async for event in workflow.run_stream(input_message):
#     if isinstance(event, WorkflowStatusEvent):
#         print(f"Status: {event.state}")
#     elif isinstance(event, WorkflowOutputEvent):
#         print(f"Output: {event.data}")

# TODO: Run and iterate on your implementation

---
## Summary & Recap

In this notebook, you learned the fundamentals of building workflows with the Microsoft Agent Framework:

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Executor** | A processing unit that receives typed messages, performs operations, and produces outputs |
| **WorkflowContext** | Provides methods (`send_message`, `yield_output`) for handlers to interact with the workflow |
| **Edges** | Define connections between executors with optional conditions for routing |
| **WorkflowBuilder** | Fluent API for constructing workflows by connecting executors |
| **Workflow Events** | Observable events (`WorkflowOutputEvent`, `ExecutorFailedEvent`, etc.) for monitoring execution |

### What You Built

1. **Basic Executors** - Class-based and function-based executors with `@handler` decorators
2. **Sequential Workflows** - Linear pipelines connecting multiple executors
3. **Concurrent Workflows** - Fan-out/fan-in patterns for parallel processing
4. **Agent Workflows** - Integrated AI agents as workflow executors
5. **Custom Agent Executors** - Wrapped agents with streaming event handling

### Key Patterns

```python
# Function-based executor (stateless - can pass directly)
@executor(id="my_executor")
async def my_step(data: str, ctx: WorkflowContext[str]) -> None:
    await ctx.send_message(data.upper())

# Simple workflow with stateless executors
workflow = (
    WorkflowBuilder()
    .add_edge(executor_a, executor_b)
    .set_start_executor(executor_a)
    .build()
)

# For agents/stateful executors - use factory functions for state isolation
def create_agent():
    return chat_client.as_agent(instructions="...", name="agent")

workflow = (
    WorkflowBuilder()
    .register_agent(factory_func=create_agent, name="agent")
    .set_start_executor("agent")
    .build()
)
```

### Next Steps

In the next notebook (`02.2-orchestrations.ipynb`), you'll learn about **pre-built orchestration patterns** including Sequential, Concurrent, and Group Chat orchestrations that simplify multi-agent coordination.